In [2]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [3]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'factory+automation' # topic of the search
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(5) # first 5 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Answering questions on the results with OpenAI GPT

In [4]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement in factory automation?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations in factory automation?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of factory automation?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

- **Title**: Adaptive Production Line Layouts

- **Explanation**: As factory automation evolves, one area ripe for innovation within the adjacent possible is the development of adaptive production line layouts. Traditional production lines are often static, designed for mass production of a single product type or variants thereof with minimal changes. However, consumer behavior shifts towards customization and the demand for a wider variety of products in smaller quantities challenge this model. Advances in robotics, AI, and modular design technologies present an opportunity to reimagine factory floors as dynamic environments. These adaptive layouts could reconfigure in real-time or near-real-time to accommodate different products, reflecting changes in consumer demand or to optimize for new manufacturing processes introduced by technological advancements. This capability would require sophisticated software for planning and simulation, advanced sensors and IoT devices for re

In [5]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., patty not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

**Problem: Adaptive Production Line Layouts**

- **Size of user group**: 3
- **Relevancy of pain point**: 3
- **How specific is the problem?**: 3

**Overall Score**: 9/9

Justification: This problem is highly relevant as most factories face the challenge of adapting to market demands quickly; it specifically targets a large and growing user group within factory automation, emphasizing the need for dynamic, flexible production systems.

---

Problem rating:

Problem: Adaptive Automation Integration

- Size of user group: 3
- Relevancy of pain point: 3
- How specific is the problem?: 3

Overall score: 9/9

Justification: Adaptive automation integration is highly relevant and specific to a large user group of factory operations managers, addressing critical challenges in integrating evolving technologies with existing systems for enhanced productivity and efficiency.

---

Problem rating:

**Problem: Seamless Data Integration**

- Size of user group: 3
- Relevancy of pain

In [6]:
# third agent to give best problem 

user_prompt3 = f"""

    Consider the following problems: {problems_list}
    Consider the corresponding ratings: {rated_problems_list}
    
   Please output the problem with the highest overall rating in the {rated_problems_list}, including the following bullet points: 
       - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 100-150 words)
        - "User group": Which user group is impacted by this problem the most (please write 10-30 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 40-50 words)
 
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt3}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )


# Print the rated problem for immediate feedback
print(completion.choices[0].message.content)
print("\n---\n")




- **Title**: Adaptive Production Line Layouts
- **Explanation**: The challenge of adaptive production line layouts centers on transitioning from static production setups, designed for mass production of similar products, to dynamic systems capable of reconfiguring in real-time or near-real-time. This shift is necessary to meet consumer demand for customization and a variety of products in smaller quantities. Achieving such flexibility requires advances in robotics, AI, modular design, and sophisticated planning software. These technologies would enable factories to adjust production lines on the fly, enhancing efficiency, reducing downtime for reconfigurations, and allowing quick response to market changes. However, this innovation introduces complexities in integration, real-time decision-making, and maintaining safety protocols during transitions.
- **User group**: Factory Managers and Operators
- **Pain point**: Factory managers and operators currently struggle with the rigidity of 

In [7]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")

def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")

if __name__ == "__main__":
    main()

1. - **Title**: Adaptive Production Line Layouts

- **Explanation**: As factory automation evolves, one area ripe for innovation within the adjacent possible is the development of adaptive production line layouts. Traditional production lines are often static, designed for mass production of a single product type or variants thereof with minimal changes. However, consumer behavior shifts towards customization and the demand for a wider variety of products in smaller quantities challenge this model. Advances in robotics, AI, and modular design technologies present an opportunity to reimagine factory floors as dynamic environments. These adaptive layouts could reconfigure in real-time or near-real-time to accommodate different products, reflecting changes in consumer demand or to optimize for new manufacturing processes introduced by technological advancements. This capability would require sophisticated software for planning and simulation, advanced sensors and IoT devices for real-time